<a href="https://colab.research.google.com/github/charu11/Natural-language-process/blob/nlp/improving_data_for_sa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
from nltk.tokenize import word_tokenize

pos = io.open('/content/drive/My Drive/natural_language_process/positive.txt', encoding='latin-1').read()
neg = io.open('/content/drive/My Drive/natural_language_process/negative.txt', encoding='latin-1').read()

documents = []

for i in pos.split('\n'):
  documents.append((i, 'pos'))

for i in neg.split('\n'):
  documents.append((i, 'neg'))

all_words = []

pos_words = word_tokenize(pos)
neg_words = word_tokenize(neg)

for w in pos_words:
  all_words.append(w.lower())

for w in neg_words:
  all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

In [0]:
import random

word_features = list(all_words.keys())[:5000]

def find_features(document):
  words = word_tokenize(document)
  features = {}
  for w in word_features:
    features[w] = (w in words)

  return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)

In [0]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from nltk.tokenize import word_tokenize


class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
        
pos = io.open('/content/drive/My Drive/natural_language_process/positive.txt', encoding='latin-1').read()
neg = io.open('/content/drive/My Drive/natural_language_process/negative.txt', encoding='latin-1').read()

for i in pos.split('\n'):
  documents.append((i, 'pos'))

for i in neg.split('\n'):
  documents.append((i, 'neg'))

all_words = []

pos_words = word_tokenize(pos)
neg_words = word_tokenize(neg)

for w in pos_words:
  all_words.append(w.lower())

for w in neg_words:
  all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:5000]

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

#print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))

featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)

# positive data example:      
training_set = featuresets[:10000]
testing_set =  featuresets[10000:]

##
### negative data example:      
##training_set = featuresets[100:]
##testing_set =  featuresets[:100]


classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

##SVC_classifier = SklearnClassifier(SVC())
##SVC_classifier.train(training_set)
##print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


voted_classifier = VoteClassifier(
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

Original Naive Bayes Algo accuracy percent: 77.52471751412429
Most Informative Features
               wonderful = True              pos : neg    =     22.6 : 1.0
              engrossing = True              pos : neg    =     22.6 : 1.0
                captures = True              pos : neg    =     20.6 : 1.0
                 lacking = True              neg : pos    =     15.3 : 1.0
                    warm = True              pos : neg    =     13.6 : 1.0
                intimate = True              pos : neg    =     13.4 : 1.0
                  unique = True              pos : neg    =     13.4 : 1.0
              unexpected = True              pos : neg    =     13.4 : 1.0
                    ride = True              pos : neg    =     13.2 : 1.0
                     van = True              neg : pos    =     12.6 : 1.0
                touching = True              pos : neg    =     12.2 : 1.0
               absorbing = True              pos : neg    =     12.1 : 1.0
            

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression_classifier accuracy percent: 80.36723163841808
SGDClassifier_classifier accuracy percent: 79.81991525423729
LinearSVC_classifier accuracy percent: 81.34710451977402
NuSVC_classifier accuracy percent: 85.36370056497175
voted_classifier accuracy percent: 82.19456214689266
